<img style="float: center; width: 100%" src="https://raw.githubusercontent.com/andrejkk/TalksImgs/master/FrontSlideUpperBan.png">
<p style="margin-bottom:2cm;"></p>


# Projekt model robota s končnimi stanji

Ime Priimek: Luka Rizman

Predmet: Uporabniku prilagojene komunikacije

Datum: Januar 2026


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## Kazalo


[Povzetek](#povzetek)

1. [Uvod](#uvod)
    1. [Opredelitev področja](#opredelitev-podrocja)
    2. [Predstavitev problema](#predstavitev-problema) 
2. [Teoretično ozadje in trenutno stanje](#teoreticno-ozadje)
    1. [Teoretični vidiki problema](#teoreticni-vidiki-problema)
    2. [Algoritmi](#algoritmi)
    3. [Merjenje uspešnosti](#merjenje-uspesnosti-resitve)
3. [Eksperimentalni del](#eksperimentalni-del)
    1. [Podatki](#podatki)
    2. [Arhitektura sistema](#arhitektura-sistema)
    3. [Demonstracija delovanja](#demonstracija-delovanja)
4. [Zaključek in razprava](#zakljucek-in-razprava)
5. [Doseženi učni izidi](#dosezeni-ucni-izidi)
6. [Literatura in viri](#literatura-in-viri)



<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## Povzetek <a class="anchor" id="povzetek"></a>

Projekt predstavlja implementacijo **robota za kognitivni trening**, ki uporablja končni avtomat za prilagajanje komunikacije glede na uporabnikovo stanje.
</br>

**1. Izhodišča:** </br>
- **Področje:** Interakcija človek-robot v kontekstu kognitivnega treninga starejših ali oseb s posebnimi potrebami.
- **Problem:** Kako robot prepozna uporabnikovo čustveno/kognitivno stanje in ustrezno prilagodi svojo komunikacijo.
- **Raziskovalno vprašanje:** Ali lahko končni avtomat, ki temelji glede na določena pravila, kako ravnati glede na stanje uporabnika, učinkovito prilagaja komunikacijo v realnem času?

**2. Rešitev:** </br>
- **Končni avtomat s 5 stanji**: Pozdrav → Razlaga → Vaja → Odmor → Zaključek
- **Pravila (Rule Engine)** na osnovi triggerjev (vedenjski signali uporabnika)
- **Klasifikacija sej** Glede na vzorce vedenja uporabnika smo naredili scenarije (mirna, zmedena, stresna, kritična seja)
- **Evalvacijski sistem** za analizo učinkovitosti prehodov med stanji

**3. Ključni rezultati:** </br>
- Delujoča Flask spletna aplikacija z interaktivno simulacijo
- Sistem za beleženje sej in interakcij v SQLite bazi
- Metrke za evalvacijo: linearnost prehodov, učinkovitost, razmerje pozitivnih/negativnih interakcij

<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 1. Uvod <a class="anchor" id="uvod"></a>

Socialni roboti so v zadnjih letih vse pogosteje uporabljeni kot podpora pri različnih oblikah pomoči in spremljanja uporabnikov (npr. v zdravstvu, izobraževanju ali pri kognitivni rehabilitaciji). Sistematični pregledi kažejo, da lahko socialno-asistivni roboti v določenih okoljih prispevajo k izboljšanju socialne interakcije, razpoloženja in zmanjšanju osamljenosti, čeprav je kakovost dokazov pogosto omejena in rezultati niso vedno enoznačni (Lihui idr. 2019).

V tem projektu je predstavljen prototip sistema, ki je usmerjen v interakcijo človek–robot skozi pogovor in vodenje uporabnika skozi strukturirane korake. Sistem je zasnovan kot simulacija: vedenjski signali niso avtomatsko prepoznani, temveč je odločanje zasnovano na vnaprej določenih pravilih. Tak pristop je skladen z “handcrafted” oziroma pravili vodenimi dialognimi pristopi, ki so enostavnejši za implementacijo, vendar imajo omejitve pri robustnosti in prilagodljivosti (Brabra idr. 2022).


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 1.1. Opredelitev področja <a class="anchor" id="opredelitev-podrocja"></a>

Področje naloge se nanaša na uporabo socialno-asistivne robotike in pogovornih sistemov kot podpore uporabnikom pri vsakdanjih nalogah ali pri specifičnih intervencijah (npr. kognitivna stimulacija, socialna podpora). V literaturi se socialno-asistivni roboti pogosto obravnavajo kot tehnologije, ki podpirajo dobrobit uporabnikov predvsem prek interakcije, vodenja in spodbujanja sodelovanja, pri čemer so učinki lahko odvisni od konteksta uporabe ter zasnove interakcije (Nichol idr. 2024).

V praksi se za tovrstne namene uporabljajo različne platforme socialnih robotov, pri čemer raziskave na področju terapije in podpore kažejo na širše zanimanje za robotsko podprte intervencije, tudi na področju avtizma in drugih razvojnih izzivov (Alabdulkareem idr. 2022).

<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 1.2. Predstavitev problema <a class="anchor" id="predstavitev-problema"></a>

Problem, ki ga obravnava naloga, je ali je **mogoča implementacija prototipa**, ki bi lahko služil kot osnova za **robota pomočnika za vodenje kognitivnega treninga za odrasle z s posebnimi problemi** v kontekstu strukturirane komunikacije in vodenja uporabnika. Ker je sistem zasnovan kot prototip, je interakcija implementirana z vnaprej določenimi pravili in prehodi med stanji pogovora, kar je tipično za končne avtomate (Brabra idr. 2022).

Imamo:
- Množico **stanj** S = {S0, S1, S2, S3, S4} (pozdrav, razlaga, vaja, odmor, zaključek)
- Množico **triggerjev** T = {t1, t2, ..., tn} (vedenjski signali uporabnika)
- Množico **intentov** I = {i1, i2, ..., im} (interpretirani nameni)
- Množico **robotskih odzivov** R = {r1, r2, ..., rk} (govorni akti)

**Cilj:** Najti preslikavo f: S × T → S' × R, ki:
1. Maksimizira uporabnikovo zadovoljstvo
2. Minimizira eskalacije (negativne interakcije)
3. Vodi sejo do uspešnega zaključka

**Pristop:**
Uporabimo **deterministični končni avtomat**, ki vključuje:
- Štetje eskalacij za avtomatski predlog zaključka
- Štetje uspešnih korakov za napredovanje
- Klasifikacijo sej v referenčne scenarije


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 2. Teoretično ozadje in trenutno stanje <a class="anchor" id="teoreticno-ozadje-in-trenutno-stanje"></a>

Za razumevanje projekta je potrebno poznavanje naslednjih konceptov:

1. **Končni avtomati (FSM)** – formalni model za opis sistemov z diskretnimi stanji
2. **Govorni akti (Speech Acts)** – kot osnovo za razumevanje namena izjav v dialogu
3. **Preprosti kriteriji uspešnosti** – kvantificirati delovanje prototipa in primerjati izide različnih interakcij

Pri pogovornih sistemih je izbira ustreznega dialognega pristopa in evalvacije pomembna, saj so uporabniške študije in metode vrednotenja osrednji del razvoja dialognih sistemov (Deriu idr. 2020).

<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.1. Teoretični vidiki problema <a class="anchor" id="teoreticni-vidiki-problema"></a>

#### Končni avtomat (Finite State Machine - FSM)

Končni avtomat je formalni model, ki opisuje sistem kot množico stanj in prehodev med njimi. V kontekstu interaktivnih sistemov (tudi v robotiki) FSM določa, pod katerimi pogoji sistem preide iz enega stanja v drugo ter katera akcija se ob tem izvede. Takšen opis je uporaben za strukturirano vodenje poteka pogovora, kjer je dialog razdeljen na zaporedne korake, ki jih sistem izvaja glede na uporabnikov vnos (Ben-Ari idr 2017).

FSM je matematični model računanja definiran kot 5-terica (S, Σ, δ, s0, F):
- S – končna množica stanj
- Σ – abeceda (množica vhodnih simbolov)
- δ: S × Σ → S – prehodna funkcija
- s0 ∈ S – začetno stanje
- F ⊆ S – množica končnih stanj

V našem primeru:
- S = {S0_GREETING, S1_EXPLANATION, S2_EXERCISE, S3_BREAK, S4_FEEDBACK}
- Σ = množica intentov (npr. "Positive affect", "User frustrated / overloaded", "Disengagement risk", "Provide action / speech feedback")
- s0 = S0_GREETING
- F = {S4_FEEDBACK}

#### Govorni akti (Speech Acts)

Govorni akti opisujejo, da pomen izjave ni zgolj v njeni vsebini, temveč tudi v namenu, ki ga govorec z izjavo opravlja (npr. vprašanje, zahteva, potrditev, obljuba). V dialognih sistemih je takšen pogled uporaben, ker omogoča interpretacijo uporabnikovega vnosa kot funkcionalnega dejanja v pogovoru, kar je povezano tudi s sodobnimi pojmi dialognih dejanj (dialogue acts) in njihovim označevanjem v standardih za analizo dialoga (Jurafsky, idr. 2025).

Robot uporablja kombinacije govornih aktov glede na situacijo:
- `Expressive + Commissive`: "Vidim, da si utrujen – naj narediva premor?"
- `Directive (clarify)`: "Ali naj ponovim navodila?"

V prototipu se govornih aktov ne prepoznava z avtomatskimi metodami, temveč se odzivi generirajo na podlagi vnaprej definiranih pravil in pričakovane strukture interakcije, kar je skladno s “handcrafted” dialognimi pristopi (Brabra idr. 2022).

### Kriteriji uspešnosti (implementirano v prototipu) <a class="anchor" id="kriteriji-uspesnosti"></a>

V implementaciji so dejansko uporabljeni naslednji kazalniki (izračunani iz logov interakcij in FSM stanja):

- **Razmerje pozitivnih interakcij (Positive Ratio)**: 
  - v UI/FSM: `positive_interactions / (positive_interactions + negative_interactions)` (pozitivno/negativno se šteje glede na množici v `core/fsm.py`).

- **Eskalacije (Total escalations)**:
  - `sum(escalation_counts.values())` (FSM vodi breakdown po intentih).

- **Linearnost prehodov (Linearity)**:
  - `forward_moves / (forward_moves + backward_moves)` glede na pričakovani vrstni red stanj `[S0, S1, S2, S3, S4]`.

- **Učinkovitost (Efficiency)**:
  - `min(1.0, min_steps / actual_steps)` kjer je `min_steps = 5`.

- **Klasifikacijska zanesljivost (Scenario match, 0–100%)**:
  - izračun ujemanja seje z referenčnimi scenariji (delno točkovanje + normalizacija na 0–100%).

- **Subjektivna ocena uporabnika (1–5)**:
  - `rating_supportive`, `rating_understandable`, `rating_non_intrusive` (shrani se po zaključku seje).

Opomba: metrike tipa **SR/CPR/PR/MR** so klasične evalvacijske formule, vendar v tej implementaciji niso neposredno izračunane, ker prototip ne označuje "pravilnih/napačnih" odgovorov, ampak analizira potek stanj, eskalacije in ujemanje s scenariji.

<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.2. Algoritmi <a class="anchor" id="algoritmi"></a>

#### Algoritem 1: Prehod med stanji končnega avtomata

```
ALGORITEM: FSM_UPDATE(current_state, inferred_intent)
VHOD: current_state ∈ S, inferred_intent ∈ I
IZHOD: next_state ∈ S

KONSTANTE:
- MAX_ESCALATIONS = 3
- MAX_SUCCESS_STEPS = 5
- FEEDBACK_INTENT = "Provide action / speech feedback"

1. should_suggest_end = FALSE
   end_reason = ""

2. IF inferred_intent ∈ POSITIVE_INTENTS:
      positive_interactions += 1
   ELSE IF inferred_intent ∈ NEGATIVE_INTENTS:
      negative_interactions += 1
      escalation_counts[inferred_intent] += 1

3. SWITCH current_state:
   CASE S0_GREETING:
      next_state = S1_EXPLANATION

   CASE S1_EXPLANATION:
      IF inferred_intent ∈ NEGATIVE_INTENTS:
         next_state = S1_EXPLANATION   // ostanemo, dodatna pojasnila
      ELSE:
         next_state = S2_EXERCISE      // pozitivno ali "drugo" (npr. Unknown)

   CASE S2_EXERCISE:
      IF inferred_intent ∉ NEGATIVE_INTENTS:
         success_steps += 1

      IF inferred_intent ∈ NEGATIVE_INTENTS:
         next_state = S3_BREAK
      ELSE IF inferred_intent == FEEDBACK_INTENT:
         next_state = S4_FEEDBACK
      ELSE IF success_steps >= MAX_SUCCESS_STEPS:
         next_state = S4_FEEDBACK
         end_reason = "success_steps"
      ELSE:
         next_state = S2_EXERCISE

   CASE S3_BREAK:
      success_steps = 0   // reset po odmoru

      IF inferred_intent ∈ POSITIVE_INTENTS:
         next_state = S2_EXERCISE
      ELSE IF inferred_intent == FEEDBACK_INTENT:
         next_state = S4_FEEDBACK
      ELSE:
         next_state = S3_BREAK

   CASE S4_FEEDBACK:
      next_state = S4_FEEDBACK

4. total_escalations = sum(escalation_counts.values())
   IF total_escalations >= MAX_ESCALATIONS AND next_state != S4_FEEDBACK:
      should_suggest_end = TRUE
      end_reason = "max_escalations"

5. state = next_state
   step_count += 1

6. RETURN next_state
```

#### Algoritem 2: Izbira pravila (Rule Selection)

```
ALGORITEM: SELECT_RULE(trigger)
VHOD: trigger ∈ T
IZHOD: rule ∈ RULES ali NULL

1. matches = {r ∈ RULES | r.Trigger == trigger}
2. IF matches = ∅:
      RETURN NULL
3. SORT matches BY priority DESC  // Critical > High > Medium > Low
4. RETURN matches[0]
```

#### Algoritem 3: Klasifikacija seje (0–100% ujemanje)

```
ALGORITEM: CLASSIFY_SESSION(interactions)
VHOD: interactions - seznam interakcij seje
IZHOD: (scenario_id, confidence)

1. stats = CALCULATE_SESSION_STATS(interactions)
   - positive_ratio, negative_ratio (glede na INTENT_CATEGORIES)
   - max_escalations (maks. escalation_count v seji)
   - final_state
   - unique_intents

2. FOR EACH scenario IN REFERENCE_SCENARIOS:
      score = 0
      max_score = 0

      IF scenario has positive_ratio_min:
         max_score += 25
         score += 25 * min(1, stats.positive_ratio / positive_ratio_min)

      IF scenario has positive_ratio_max:
         max_score += 25
         score += partial (nižja točka, če je ratio previsok)

      IF scenario has max_escalations:
         max_score += 25
         score += partial (nižja točka, če je eskalacij preveč)

      IF scenario has min_escalations:
         max_score += 25
         score += 25 if stats.max_escalations >= min_escalations

      IF scenario has expected_final_state:
         max_score += 25
         score += 25 if stats.final_state == expected_final_state

      IF scenario has typical_intents:
         max_score += 25
         score += 25 * (|intersection| / |typical_intents|)

      confidence = (score / max_score) * 100

3. RETURN scenario z največjim confidence
```

**Uporabljene knjižnice:**
- Flask 3.0 – spletni okvir
- Flask-SQLAlchemy 3.1 – ORM za SQLite bazo
- Python dataclasses – za definicijo FSM strukture


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.3. Merjenje uspešnosti rešitve <a class="anchor" id="merjenje-uspesnosti-resitve"></a>

Za evalvacijo sistema uporabljamo več metrik (izračun v `core/fsm.py` in `evaluation/functions.py`):

#### 1. Razmerje pozitivnih interakcij

**Positive Ratio** = |pozitivne interakcije| / |pozitivne + negativne|

Opomba: v trenutni implementaciji se pozitivno/negativno štetje v UI/FSM izvaja na podlagi množic intentov v `core/fsm.py` (npr. "Positive affect" je pozitivno; "User frustrated / overloaded" in "Disengagement risk" sta negativna).

#### 2. Linearnost prehodov

**Linearity** = forward_moves / (forward_moves + backward_moves)

Meri, kako direkten je bil prehod skozi stanja glede na pričakovani vrstni red `[S0, S1, S2, S3, S4]`.

#### 3. Učinkovitost (Efficiency)

**Efficiency** = min(1.0, min_steps / actual_steps)

kjer je `min_steps = 5` (minimalno potrebnih korakov za zaključek, če je zaključek dosežen).

#### 4. Klasifikacijska zanesljivost

Ujemanje seje z referenčnim scenarijem (0-100%):
- ≥ 70%: Visoka zanesljivost
- 50-70%: Srednja zanesljivost  
- < 50%: Nizka zanesljivost

#### Referenčni scenariji (implementirani)

| Scenarij | Opis | Uporabljene značilnosti v ujemanju |
|----------|------|-------------------------------------|
| **Mirna seja** | Uporabnik pozitiven in sodelujoč | positive_ratio ≥ 0.7, max_escalations ≤ 1, expected_final_state = S4_FEEDBACK |
| **Pogosto zmeden** | Potrebuje več ponovitev | positive_ratio 0.3–0.6, max_escalations ≤ 3, typical_intents (če se ujemajo) |
| **Odvrača pozornost** | Pogosto gleda drugam | positive_ratio ≥ 0.4, max_escalations ≤ 2, typical_intents (če se ujemajo) |
| **Pod stresom** | Znaki stresa in preobremenjenosti | positive_ratio ≤ 0.4, min_escalations ≥ 2, typical_intents (če se ujemajo) |
| **Kritična seja** | Veliko eskalacij | positive_ratio ≤ 0.3, min_escalations ≥ 3 |

Opomba: scenariji vsebujejo tudi polja (npr. `expected_triggers`), ki so v dokumentaciji uporabna za razlago, vendar se v izračunu ujemanja trenutno ne uporabljajo.


</br>
<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 3. Eksperimentalni del <a class="anchor" id="eksperimentalni-del"></a>

### Navodila za zagon aplikacije

**1. Namestitev odvisnosti:**
```bash
pip install -r requirements.txt
```

**2. Zagon aplikacije:**
```bash
python app.py
```


**Potrebne knjižnice (requirements.txt):**
```
Flask==3.0.0
Flask-SQLAlchemy==3.1.1
pymysql==1.1.0
python-dotenv==1.0.1
```

**Struktura projekta:**
```
RobotKoncniAvtomat/
├── app.py              # Glavna Flask aplikacija
├── config.py           # Konfiguracija
├── core/               # Jedro sistema
│   ├── fsm.py          # Končni avtomat
│   └── rules_loader.py # Pravila
├── db/                 # Baza podatkov
│   └── models.py       # SQLAlchemy modeli
├── evaluation/         # Evalvacijski sistem
│   ├── functions.py    # Evalvacijske funkcije
│   ├── categories.py   # Kategorije intentov
│   └── scenarios.py    # Referenčni scenariji
├── routes/             # API endpoints
├── templates/          # HTML predloge
└── static/             # CSS, JS
```


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.1. Podatki <a class="anchor" id="podatki"></a>

#### Pravila (Rules)

Sistem uporablja 14 pravil definiranih v `core/rules_loader.py`. Vsako pravilo vsebuje:

| Polje | Opis | Primer |
|-------|------|--------|
| **Trigger** | Vedenjski signal uporabnika (UI gumb) | "User face stressed" |
| **Inferred Intent** | Interpretiran namen | "User frustrated / overloaded" |
| **Priority** | Prioriteta (Critical/High/Medium/Low) | "Medium" |
| **ConfidenceThrs.** | Prag zaupanja | 0.60 |
| **Escalated Action** | Eskalacijska akcija (lahko `None`) | `None` |
| **escalationCount** | Polje za štetje eskalacij (v pravilih je lahko `None`; dejansko štetje vodi FSM) | `None` |
| **Robot Speech Act** | Tip govornega akta | "Expressive + Commissive" |
| **RobotText** | Robotov odziv | "You look stressed — shall I slow down?" |

Opomba: v tej verziji prototipa je seznam pravil statično definiran v Python datoteki (ne bere se dinamično iz Excel datoteke).

#### Primeri pravil:

```python
{
    "Trigger": "User face stressed",
    "Inferred Intent": "User frustrated / overloaded",
    "Priority": "Medium",
    "ConfidenceThrs.": 0.60,
    "Escalated Action": None,
    "escalationCount": None,
    "Robot Speech Act": "Expressive + Commissive",
    "RobotText": "You look stressed — shall I slow down?"
},
{
    "Trigger": "User smiles/laughs",
    "Inferred Intent": "Positive affect",
    "Priority": "Medium",
    "ConfidenceThrs.": 0.60,
    "Escalated Action": None,
    "escalationCount": None,
    "Robot Speech Act": "Expressive",
    "RobotText": "Haha, that's funny!"
}
```

#### Baza podatkov

SQLite baza (`robot_fsm.db`) vsebuje dve tabeli:

**Tabela `sessions`:**
- id, started_at, ended_at
- rating_supportive, rating_understandable, rating_non_intrusive (1-5)
- evaluated_at

**Tabela `interactions`:**
- id, session_id, step_number, timestamp
- state_before, state_after
- trigger, inferred_intent
- robot_speech_act, robot_utterance
- priority, escalation_count


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.2. Arhitektura sistema <a class="anchor" id="arhitektura-sistema"></a>

#### Diagram stanj FSM

```
                    ┌──────────────┐
                    │ S0_GREETING  │
                    │   👋 Pozdrav │
                    └──────┬───────┘
                           │ vedno
                           ▼
                    ┌──────────────────┐
            ┌───────│ S1_EXPLANATION   │◄──────┐
            │       │   📖 Razlaga     │       │
            │       └────────┬─────────┘       │ negativen
            │                │                 │ (pojasnila)
            │                │ pozitiven ali   │
            │                │ 2+ korakov      │
            │                ▼                 │
            │       ┌──────────────────┐       │
            │   ┌──►│   S2_EXERCISE    │───────┘
            │   │   │    🎯 Vaja       │
            │   │   └────────┬─────────┘
            │   │            │
            │   │  pozitiven │ negativen
            │   │  nevtralen │
            │   │            ▼
            │   │   ┌──────────────────┐
            │   └───│    S3_BREAK      │◄──┐
            │       │    ☕ Odmor      │   │ negativen
            │       └────────┬─────────┘───┘
            │                │ feedback / 
            │                │ success_steps >= 5
            │                ▼
            │       ┌──────────────────┐
            └──────►│   S4_FEEDBACK    │
                    │   ✅ Zaključek   │
                    └──────────────────┘
```

**Legenda prehodov:**
| Iz stanja | Signal | V stanje |
|-----------|--------|----------|
| S0_GREETING | vedno | S1_EXPLANATION |
| S1_EXPLANATION | pozitiven | S2_EXERCISE |
| S1_EXPLANATION | negativen | S1_EXPLANATION (pojasnila) |
| S1_EXPLANATION | nevtralen/feedback (2+ korakov) | S2_EXERCISE |
| S2_EXERCISE | negativen | S3_BREAK |
| S2_EXERCISE | success_steps >= 5 | S4_FEEDBACK |
| S3_BREAK | pozitiven ali nevtralen | S2_EXERCISE |
| S3_BREAK | negativen | S3_BREAK |
| S3_BREAK | feedback | S4_FEEDBACK |
| S4_FEEDBACK | vedno | S4_FEEDBACK (končno) |

#### Arhitektura aplikacije

```
┌─────────────────────────────────────────────────────────┐
│                    FLASK APP                            │
├─────────────────────────────────────────────────────────┤
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   │
│  │   routes/   │   │    core/    │   │    db/      │   │
│  │  main.py    │──►│   fsm.py    │   │  models.py  │   │
│  │ evaluate.py │   │rules_loader │   │             │   │
│  └─────────────┘   └─────────────┘   └─────────────┘   │
│         │                │                  │          │
│         │                │                  │          │
│         ▼                ▼                  ▼          │
│  ┌─────────────────────────────────────────────────┐   │
│  │              evaluation/                        │   │
│  │  functions.py │ categories.py │ scenarios.py   │   │
│  └─────────────────────────────────────────────────┘   │
├─────────────────────────────────────────────────────────┤
│  templates/        static/css/         static/js/      │
│  index.html        main.css            main.js         │
│  evaluate.html     evaluate.css        evaluate.js     │
└─────────────────────────────────────────────────────────┘
```

#### API Endpoints

| Metoda | Endpoint | Opis |
|--------|----------|------|
| GET | `/` | Glavna stran simulacije |
| POST | `/trigger` | Obdelava triggerja (ustvari DB sejo ob prvem triggerju) |
| POST | `/reset` | Ponastavitev seje (zaključi/počisti trenutno) |
| POST | `/force-end` | Prisili zaključek |
| GET | `/statistics` | Statistika trenutne seje |
| GET | `/evaluate` | Stran za evalvacijo |
| GET | `/api/all-sessions` | Seznam vseh (ne-praznih) sej |
| GET | `/api/session/<id>` | Podrobnosti seje + funkcionalna evalvacija |
| POST | `/api/evaluate-session` | Shrani subjektivno oceno uporabnika (1–5) |
| GET | `/api/scenarios` | Vrne seznam referenčnih scenarijev |


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.3. Demonstracija delovanja <a class="anchor" id="demonstracija-delovanja"></a>

#### Primer mirne seje

**Potek:**
1. `greet` → S0→S1 → "Hello! Nice to see you!"
2. `User smiles/laughs` → S1→S2 → "Haha, that's funny!"
3. `User smiles/laughs` → S2→S2 → "Haha, that's funny!"
4. `end of user speech` → S2→S2 → "I understand. Let me continue..."
5. `User smiles/laughs` → S2→S4 (5 uspešnih korakov) → zaključek

**Rezultat evalvacije:**
- Scenarij: "Mirna seja" 
- Confidence: 85%
- Positive ratio: 80%
- Linearnost: 100%

#### Primer stresne seje

**Potek:**
1. `greet` → S0→S1 → "Hello! Nice to see you!"
2. `User face stressed` → S1→S1 → "You look stressed — shall I slow down?"
3. `User leans back / crosses arms` → S1→S2→S3 → "Should we take a short break?"
4. `User face stressed` → S3→S3 → "You look stressed — shall I slow down?"
5. (3 eskalacije) → robot predlaga zaključek

**Rezultat evalvacije:**
- Scenarij: "Pod stresom" 
- Confidence: 78%
- Positive ratio: 20%
- Eskalacije: 3

#### Statistika seje (primer)

```json
{
  "step_count": 8,
  "positive_interactions": 5,
  "negative_interactions": 2,
  "total_escalations": 2,
  "success_steps": 4,
  "current_state": "S2_EXERCISE",
  "is_final": false,
  "positive_ratio": 0.71,
  "should_suggest_end": false
}
```


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 4. Zaključek in razprava <a class="anchor" id="zakljucek-in-razprava"></a>

### Glavni rezultati

1. **Uspešno implementiran končni avtomat** za vodenje interakcije človek-robot s 5 stanji in logiko prehodov
2. **Rule Engine** z 14 pravili za izbiro ustreznega robotskega odziva
3. **Evalvacijski sistem** za klasifikacijo sej v 5 referenčnih scenarijev
4. **Interaktivna spletna aplikacija** za testiranje in demonstracijo

### Zaključki

Pristop končnega avtomata je primeren za modeliranje dialogov v robot-človek komunikaciji. Sistem pravil omogoča prilagodiljivost in enostavno razširjanje. Evalvacijske metrike dajejo uporabne povratne informacije o kvaliteti sej. Klasifikacija scenarijev omogoča primerjavo med različnimi sejami.

### Omejitve
Sistem je zasnovan kot simulacija in zato ne vključuje dejanskega prepoznavanja vedenjskih signalov, temveč deluje na vnaprej določenih vhodih. Pravila, ki usmerjajo njegovo delovanje, so statična, kar pomeni, da ne uporablja strojnega učenja ali prilagajanja za postopno izboljševanje odzivov. 

### Prihodnje delo

V prihodnosti se svetuje integracija z računlaniškim vidom torej za dejansko detekcijo obraznih mimik in kretenj (kamera). Dodalo bi se lahko tudi več trigerjev ter več stanj za izdelavo modela strojnega učenja ter morda večjezična podpora. 


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 5. Doseženi učni izidi <a class="anchor" id="dosezeni-ucni-izidi"></a>

Pri izdelavi projekta sem poglobil razumevanje **teoretičnih konceptov**, kot so končni avtomati za modeliranje sistemov z diskretnimi stanji, govorni akti kot osnova za razumevanje vrst izjav v komunikaciji ter temeljni principi interakcije človek–robot in prilagajanja komunikacije uporabniku. 

Poleg tega sem spoznal **algoritmične pristope**, predvsem implementacijo determinističnega končnega avtomata v Pythonu, uporabo rule-based sistemov za izbiro odzivov na podlagi vnaprej določenih pravil ter uporabo evalvacijskih metrik za analizo uspešnosti interakcij. 

Pri praktičnem delu sem pridobil tudi izkušnje z **različnimi tehnologijami**, med drugim z uporabo spletnega ogrodja Flask, ORM-ja SQLAlchemy za delo z bazami podatkov, strukturiranjem projekta v ločene module ter načrtovanjem REST API-jev za spletne aplikacije.



<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 6. Literatura in viri  <a class="anchor" id="literatura-in-viri"></a>

### Literatura


Alabdulkareem, A., Alhakbani, N., & Al-Nafjan, A. (2022).  
*A systematic review of research on robot-assisted therapy for children with autism*. **Sensors, 22**(3), 944.  
https://www.mdpi.com/1424-8220/22/3/944

Ben-Ari, M., & Mondada, F. (2018).  
Finite state machines. In *Elements of robotics*. Springer, Cham.  
https://link.springer.com/chapter/10.1007/978-3-319-62533-1_4

Brabra, H., Baez, M., Benatallah, B., Gaaloul, W., Bouguelia, S., & Zamanirad, S. (2022).  
Dialogue management in conversational systems: A review of approaches, challenges, and opportunities.  
**IEEE Transactions on Cognitive and Developmental Systems, 14**(3), 783–798.  
https://ieeexplore.ieee.org/document/9447005

Deriu, J., Rodrigo, A., Otegi, A., Echegoyen, G., Rosset, S., Agirre, E., & Cieliebak, M. (2021).  
Survey on evaluation methods for dialogue systems.  
**Artificial Intelligence Review, 54**, 755–810.  
https://link.springer.com/article/10.1007/s10462-020-09866-x

Jurafsky, D., & Martin, J. H. (2024).  
*Speech and language processing: An introduction to natural language processing, computational linguistics, and speech recognition* (3rd ed., draft).  
Stanford University.  
https://web.stanford.edu/~jurafsky/slp3/

Nichol, B., McCready, J., Erfani, G., Comparcini, D., Simonetti, V., Cicolini, G., Mikkonen, K., Yamakawa, M., & Tomietto, M. (2024).  
Exploring the impact of socially assistive robots on health and wellbeing across the lifespan: An umbrella review and meta-analysis.  
**International Journal of Nursing Studies, 153**, 104730.  
https://www.sciencedirect.com/science/article/pii/S0020748924000427?via%3Dihub



### Tehnicna dokumentacija

6. **Flask Documentation.** https://flask.palletsprojects.com/
   - Uradna dokumentacija Flask okvira.

7. **SQLAlchemy Documentation.** https://docs.sqlalchemy.org/
   - Dokumentacija SQLAlchemy ORM.

8. **Python dataclasses.** https://docs.python.org/3/library/dataclasses.html
   - Dokumentacija Python dataclasses modula.



<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">